In [1]:
import sklearn.utils
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import csv
from bs4 import BeautifulSoup
import requests

Using TensorFlow backend.


In [2]:
def loader(filename):
    file=open(filename, "r")
    lines=csv.reader(file)
    dataset=list(lines)
    return dataset

In [3]:
url='https://karki23.github.io/Weather-Data/assignment.html'
sauce=requests.get(url)
srccode=BeautifulSoup(sauce.content, "html.parser")
new_file=open("datasets\\all_data.csv", "w", newline="")
df=pd.read_csv("datasets\\albury.csv")
writer=csv.writer(new_file)
writer.writerow(list(df.keys()))

248

In [4]:
all_cities=srccode.find_all('a')
for i in all_cities:
    s=i.get('href')[0:len(i)-5:]
    file_name="datasets\\"+s+"csv"
    dataset=loader(file_name)
    
    train_set=dataset[1:]   
    for i in train_set:
        writer.writerow(i)
new_file.close()

In [5]:
df = pd.read_csv('datasets\\all_data.csv')
df=sklearn.utils.shuffle(df)
df=df.drop(columns=['Date', 'Location', 'RISK_MM'])

In [6]:
df['WindGustDir'] = pd.Categorical(df['WindGustDir'])
df['WindGustDir'] = df.WindGustDir.cat.codes
df['WindDir9am'] = pd.Categorical(df['WindDir9am'])
df['WindDir9am'] = df.WindDir9am.cat.codes
df['WindDir3pm'] = pd.Categorical(df['WindDir3pm'])                 #Line 44-53 converts specific column values to descrete numerical values
df['WindDir3pm'] = df.WindDir3pm.cat.codes
df['RainToday'] = pd.Categorical(df['RainToday'])
df['RainToday'] = df.RainToday.cat.codes
df['RainTomorrow'] = pd.Categorical(df['RainTomorrow'])
df['RainTomorrow'] = df.RainTomorrow.cat.codes

In [7]:
df[['WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']] = df[['WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']].astype(float)               # This line converts all datatypes to int so that the entire file is made up of single datatype


In [8]:
for i in df.keys():
    df[i].fillna(df[i].mean(), inplace=True)                #Handling missing/NaN values by replacing them with the mean value of each column


In [9]:
data = df.drop(columns=['RainTomorrow'])
target = df[['RainTomorrow']]
x_train=data[1: int(0.8*len(data))]
y_train=target[1:int(0.8*len(target))]
x_test=data[int((0.8*len(data))): ]
y_test=target[int((0.8*len(data))): ]

In [10]:
model=Sequential()
n_cols=data.shape[1]
model.add(Dense(21, activation='sigmoid', input_shape=(n_cols,)))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(5, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

W0724 12:50:54.174362 32080 deprecation_wrapper.py:119] From C:\Users\Prajwal\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0724 12:50:54.194334 32080 deprecation_wrapper.py:119] From C:\Users\Prajwal\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0724 12:50:54.202324 32080 deprecation_wrapper.py:119] From C:\Users\Prajwal\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0724 12:50:54.266272 32080 deprecation_wrapper.py:119] From C:\Users\Prajwal\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0724 12:50:54.290230 32080 deprecation_wrapper.py:119] From C:\Users\Pr

In [11]:
model.fit(x_train  , y_train, epochs=5)  


W0724 12:50:57.155772 32080 deprecation_wrapper.py:119] From C:\Users\Prajwal\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/5
113753/113753 [==============================] - 4s 32us/step - loss: 0.4326 - acc: 0.8106
Epoch 2/5
113753/113753 [==============================] - 3s 28us/step - loss: 0.4012 - acc: 0.8324
Epoch 3/5
113753/113753 [==============================] - 3s 28us/step - loss: 0.3969 - acc: 0.8334
Epoch 4/5
113753/113753 [==============================] - 3s 29us/step - loss: 0.3995 - acc: 0.8323
Epoch 5/5
113753/113753 [==============================] - 3s 30us/step - loss: 0.3964 - acc: 0.8347


In [12]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print("\nTHE TEST ACCURACY IS : ", test_acc)


28439/28439 [==============================] - 0s 15us/step

THE TEST ACCURACY IS :  0.840184254025754
